This file should only be used to contain the code used to download data by using the selenium 

and

Upload data to minio

### Download file from drive

In [2]:
from selenium import webdriver
import glob, os, time
import chromedriver_binary  # Adds chromedriver binary to path
from selenium.webdriver.common.by import By

download_url = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing"

options = webdriver.ChromeOptions()

# Set download path for data.zip
prefs = {"download.default_directory": "./"}
options.add_experimental_option("prefs", prefs)
options.add_argument("--headless")
# Must be full size screen to see download button
options.add_argument("--window-size=1920,1080")

# Delete any file with .zip post-fix
for f in glob.glob("data*.zip"):
    os.remove(f)

driver = webdriver.Chrome(options=options)
driver.get(download_url)
# Wait for web fully loaded
driver.implicitly_wait(1)

download_link = driver.find_element(By.CSS_SELECTOR, "div.akerZd").find_element(By.CSS_SELECTOR, "span")
download_link.click()

# Wait for download complete for TIME_LIMIT second
TIME_LIMIT = 30
now = time.time()
while True:
    file = glob.glob("data.zip")
    if file:
        break
    else:
        # Break if out of limit
        if time.time() - now > TIME_LIMIT:
            break
        time.sleep(0.5)
driver.close()


In [1]:
import zipfile, shutil
if os.path.exists("data-raw"):
    shutil.rmtree("data-raw")
os.mkdir("./data-raw")
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data-raw")


NameError: name 'os' is not defined

In [4]:
# Define S3 storage
obj_storage_access_key = os.getenv('MINIO_ACCESS_KEY')
obj_storage_secret_key = os.getenv('MINIO_SECRET_KEY')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://minio:9000')


In [8]:
from ultils import upload_file_to_minio
# Upload file to minio
upload_file_to_minio('data-raw/data.json', 'data', 'data-raw/data.json', obj_storage_endpoint, obj_storage_access_key, obj_storage_secret_key)
upload_file_to_minio('data-raw/data2.json', 'data', 'data-raw/data2.json', obj_storage_endpoint, obj_storage_access_key, obj_storage_secret_key)
upload_file_to_minio('data-raw/data3.json', 'data', 'data-raw/data3.json', obj_storage_endpoint, obj_storage_access_key, obj_storage_secret_key)
